# YeaZ evaluation

Evaluates the YeaZ algorithm in a similar manor as the Mask-RCNN pipeline.

We've precomputed the output of YeaZ for the YIT data and it's downloaded as `YeaZ_results.zip`.

In [1]:
import numpy as np
import pandas as pd
from google.colab import files 
import matplotlib.pyplot as plt
%matplotlib notebook

try:
  from download import download
except ImportError:
  !pip3 install download
  from download import download

try:
    # raise ImportError()
    from yeastcells import data, clustering, features, visualize, evaluation
except ImportError:
    !test -e yeastcells-detection-maskrcnn || git clone https://github.com/ymzayek/yeastcells-detection-maskrcnn.git
    !cd yeastcells-detection-maskrcnn; git pull origin main
    !pip3 install ./yeastcells-detection-maskrcnn
    from yeastcells import data, clustering, features, visualize, evaluation

/usr/local/lib/python3.7/dist-packages/skimage/io/manage_plugins.py:23: UserWarning: Your installed pillow version is < 7.1.0. Several security issues (CVE-2020-11538, CVE-2020-10379, CVE-2020-10994, CVE-2020-10177) have been fixed in pillow 7.1.0 or higher. We recommend to upgrade this library.
  from .collection import imread_collection_wrapper


## Downloads

Downloads ground truth and YeaZ results

In [ ]:
pipeline_path = f'/content/yeastcells-detection-maskrcnn'

download(
    'https://datascience.web.rug.nl/YeaZ_results.zip',
    f'{pipeline_path}/YeaZ_results/YeaZ_results.zip'
)

download(
    'https://datascience.web.rug.nl/YIT-Benchmark2.zip', 
    f'{pipeline_path}/YIT-Benchmark2/YIT-Benchmark2.zip'
)

!cd '{pipeline_path}' && unzip -o 'YIT-Benchmark2/YIT-Benchmark2.zip' -d YIT-Benchmark2
!cd '{pipeline_path}' && unzip -o 'YeaZ_results/YeaZ_results.zip' -d YeaZ_results

## YeaZ results

Prepares the YeaZ results for evaluation

In [3]:
testset = 'TestSet1' 
# one mask per frame, with integer encoded segmentation.
masks = data.read_tiff_mask(f'{pipeline_path}/YeaZ_results/YeaZ_masks_{testset}.tiff')

# a dataframe with every detection per frame seperately
yeaz_detections = pd.DataFrame([
  {'frame': frame, 'cell': cell}
  for frame, mask in enumerate(masks)
  for cell in np.unique(mask)
  if cell > 0
])

# create one-hot encoded masks, like we get from the Mask-RCNN, such that they
# fit our evaluation pipeline.
masks = (
    masks[yeaz_detections['frame']] # repeat frames per detection of a frame
    == yeaz_detections['cell'].values[:, None, None] # one-hot encode
)
# we could use the dataframe's index, but a seperate column is safer to mutations
yeaz_detections['mask'] = np.arange(len(masks))

## Ground truth

The ground truth consists of annotations of
 * `frame`: frame number starting at 1,
 * `cell`: tracks cells over frames, and
 * `x`, `y`: cell position.

In [4]:
ground_truth = pd.read_csv(
    f'{pipeline_path}/YIT-Benchmark2/{testset}/GroundTruth/GroundTruth_Segmentation.csv',
    sep=', ' # there's an awkward space next to the commas
).drop('Cell_colour', axis=1).rename(
    {'Frame_number': 'frame',	'Cell_number': 'cell',
     'Position_X' :'x', 'Position_Y': 'y'}, axis=1).reindex()

# detections start counting at 0, ground truths at 1. rectify:
ground_truth['frame'] = ground_truth['frame'] - 1

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:3: ParserWarning: Falling back to the 'python' engine because the 'c' engine does not support regex separators (separators > 1 char and different from '\s+' are interpreted as regex); you can avoid this warning by specifying engine='python'.
  This is separate from the ipykernel package so we can avoid doing imports until


## Evaluation

Evaluates the YeaZ output similar to the Mask-RCNN

### Results

In [9]:
segmentation_metrics = evaluation.calculate_metrics(
    evaluation.get_segmention_metrics(ground_truth, yeaz_detections, masks))
tracking_metrics = evaluation.calculate_metrics(
    evaluation.get_tracking_metrics(ground_truth, yeaz_detections, masks))

metrics = [
    {'task': task, 'metric': metric, 'value': value}
    for task, result in {
        'tracking': tracking_metrics,
        'segmentation': segmentation_metrics}.items()
    for metric, value in result.items()
]
results = pd.DataFrame(metrics)

results.pivot('task', columns=['metric'], values='value')

metric,Accuracy,F1-score,Precision,Recall
task,,,,
segmentation,0.655374,0.791814,0.669851,0.968076
tracking,0.666061,0.799563,0.682609,0.964881


### Download

In [10]:
results.pivot('task', columns=['metric'], values='value').to_excel('yeaz-performance.xlsx')

try:
  from google.colab.files import download
  download('yeaz-performance.xlsx')
except ImportError:
  print('Not on Google Colab, won\'t start download')

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>